
Interactive theorem proving is cool.

Automated theorem proving is really cool.

It does happen though that you can get outside the ability of an automated theorem prover in one shot. It may need breadcrumbs or guidance along the way.
But then you need some kind of scaffolding to organize these hints.

I think this is an interesting design I don't see much.

The basic premise is that we allow a single inference rule SUPERMODUS (TM) which uses an automated prover to discharge a big step of the proof. We get to manually select if we like the hypothesies it has access to.




In [2]:
from typing import Any, Tuple, List
from z3 import *
Form = Any
Thm = Tuple[int, Form]
BoolRef.__and__ = lambda self, other: And(self, other)
BoolRef.__or__ = lambda self, other: Or(self, other)
BoolRef.__invert__ = lambda self: Not(self)
BoolRef.__gt__ = lambda self, other: Implies(self, other)

def check(thm : Thm):
    hsh, form = thm
    assert hsh == hash(("secret",form))

# recording all known theorems makes certain big hammer autos possible.
# thms = [] # or set()

def trust(form : Form) -> Thm:
    #thms.append(concthm)
    return hash(("secret",form)), form

def infer(hyps : List[Thm], conc : Form, timeout=1000) -> Thm:
    s = Solver()
    for hyp in hyps:
        check(hyp)
        s.add(hyp[1])
    s.add(Not(conc))
    s.set("timeout", timeout)
    res = s.check()
    if res != z3.unsat:
        print(s.sexpr())
        if res == z3.sat:
            print(s.model())
        assert False,res

    return trust(conc)

def extend(hyps, x,y, P : Form, prefix="c"):
    infer(hyps, ForAll([x],Exists([y], P)))
    f = FreshConst(ArraySort(x.sort(), y.sort()), prefix=prefix)
    return f, trust(ForAll([x], P.substitute(y,f[y])))

# A primitive definition mechanism. Maybe + Lambda this is good enough?
def define(name, x):
    f = FreshConst(x.sort(), prefix=name)
    return f, trust(f == x)




# Tactics

There are a number of derived rules. We can reuse z3's tactics. This is a bit of slight of hand.

In [47]:
def exI(x, P : Thm) -> Thm:
    return infer([P], Exists([x], P[1]))

A = DeclareSort("A")
a = Const("a", A)
P = Function("P", A, BoolSort())
ax = trust(P(a))
exI(a, ax)


# maybe
def simp(thm : Thm):
    check(thm)
    return infer([thm],z3.simplify(thm[1]))

x = Int("x")
thm = infer([], x + 0 == x)
simp(thm)

# other tactics
def tactic(name, thm):
    g = Goal(thm[1])
    conc = z3.Tactic(name)(g)
    print(conc)
    return infer([thm], conc)

tactic("simplify", thm)
    


[[]]


Z3Exception: True, False or Z3 Boolean expression expected. Received [[]] of type <class 'z3.z3.ApplyResult'>

In [46]:
z3.tactics()

['ackermannize_bv',
 'subpaving',
 'horn',
 'horn-simplify',
 'nlsat',
 'qfnra-nlsat',
 'qe-light',
 'nlqsat',
 'qe',
 'qsat',
 'qe2',
 'qe_rec',
 'psat',
 'sat',
 'sat-preprocess',
 'ctx-solver-simplify',
 'psmt',
 'unit-subsume-simplify',
 'aig',
 'add-bounds',
 'card2bv',
 'degree-shift',
 'diff-neq',
 'eq2bv',
 'factor',
 'fix-dl-var',
 'fm',
 'lia2card',
 'lia2pb',
 'nla2bv',
 'normalize-bounds',
 'pb2bv',
 'propagate-ineqs',
 'purify-arith',
 'recover-01',
 'bit-blast',
 'bv1-blast',
 'bv_bound_chk',
 'propagate-bv-bounds',
 'propagate-bv-bounds2',
 'reduce-bv-size',
 'bv-slice',
 'bvarray2uf',
 'dt2bv',
 'elim-small-bv',
 'max-bv-sharing',
 'blast-term-ite',
 'cofactor-term-ite',
 'collect-statistics',
 'ctx-simplify',
 'demodulator',
 'der',
 'distribute-forall',
 'dom-simplify',
 'elim-term-ite',
 'elim-uncnstr2',
 'elim-uncnstr',
 'elim-predicates',
 'euf-completion',
 'injectivity',
 'snf',
 'nnf',
 'occf',
 'pb-preprocess',
 'propagate-values2',
 'propagate-values',
 'reduc

In [43]:
x = Bool("x")
print( x & x > ~x | x)
x = Array("x", StringSort(), StringSort())
print(x.sort())
print(x.sort().domain())
print(x.sort().range())
print(dir(x.sort()))
x.sort().name()

Implies(And(x, x), Or(Not(x), x))
Array(String, String)
String
String
['__bool__', '__class__', '__copy__', '__deepcopy__', '__del__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__nonzero__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_repr_html_', 'as_ast', 'ast', 'cast', 'ctx', 'ctx_ref', 'domain', 'domain_n', 'eq', 'get_id', 'hash', 'kind', 'name', 'range', 'sexpr', 'subsort', 'translate', 'use_pp']


'Array'

In [ ]:
class Backward():
    def __init__(self, goal):
        self.goal = infer([], goal > goal)
    def intros(self):
        pass
class Forward(): #Isar
    def __init__(self, goal):
        self.goal = goal
        self.lemmas = []
        self.vars = []
        self.hyps = []
    def intro(self,vars):
        self.vars.extend(vars)
        return self
    def assume(self, hyps):
        self.hyps.extend(hyps)
        return self
    def wrap(self, form):
        return ForAll(self.vars, Implies(And(self.hyps), form))
    def have(self, conc, by=[]):
        self.lemmas.append(infer(by + self.lemmas, conc))
        return self
    def qed(self):
        return infer(self.lemmas, self.goals)
    
class Calc():
    def __init__(self, lhs, rhs):
        self.lhs = [lhs]
        self.lhsthm = infer([], lhs == lhs)
        self.rhs = [rhs]
    def then(self, rhs, by=[]): # step? rw?
        self.thm = infer(by + [self.lhsthm], self.rhs == rhs)
    def simp():




# Peano Arithmetic


In [27]:
# Z3py adt of natural numbers
Nat = Datatype("Nat")
Nat.declare("zero")
Nat.declare("succ", ("pred", Nat))
Nat = Nat.create()
print(Nat.succ(Nat.zero))

# Peano
def induct(P : Form) -> Thm:
    print(P.sort())
    assert P.sort().name() == "Array"
    assert P.sort().domain() == Nat
    assert P.sort().range() == BoolSort()
    n = FreshConst(Nat)
    hyp = P[Nat.zero] & ForAll([n], P[n] > P[Nat.succ(n)])
    #------------------------------------------
    conc =  ForAll([n], P[n])
    return trust(hyp > conc)

x,y = Consts("x y", Nat)
add = Function("add", Nat, Nat, Nat)
zero_add = trust(ForAll([x], add(Nat.zero, x) == x))
succ_add = trust(ForAll([x,y], add(Nat.succ(x), y) == Nat.succ(add(x, y))))

P = Lambda([x], add(x,Nat.zero) == add(Nat.zero,x))

base = modus([], P[Nat.zero])
ind = modus([zero_add, succ_add], ForAll([x], P[x] > P[Nat.succ(x)]))
add_zero = modus([base,ind,induct(P)], ForAll([x], P[x]))
add_zero_prime = modus([zero_add, succ_add, induct(P)], ForAll([x], P[x]))


P = Lambda([x], ForAll([y], add(x,Nat.succ(y)) == Nat.succ(add(x,y))))
add_succ = modus([zero_add,succ_add, induct(P)], ForAll([x],P[x]))
comm_add = modus([zero_add, succ_add, add_zero, add_succ, induct(Lambda([x], ForAll([y], add(x,y) == add(y,x))))] ,
                  ForAll([x,y], add(x, y) == add(y, x)))

comm_add


succ(zero)
Array(Nat, Bool)
Array(Nat, Bool)
Array(Nat, Bool)
Array(Nat, Bool)


(7898931013983239133, ForAll([x, y], add(x, y) == add(y, x)))

In [ ]:
def induct_int(P : Form) -> Thm:
    assert P.sort().name() == "Array"
    assert P.sort().domain() == IntSort()
    assert P.sort().range() == BoolSort()
    n = FreshConst(IntSort())
    hyp = P[0] & ForAll([n], P[n] > P[n - 1] & P[n + 1])
    #------------------------------------------
    conc =  ForAll([n], P[n])
    return trust(hyp > conc)
# https://math.stackexchange.com/questions/2659184/can-induction-be-done-to-prove-statements-for-integers


def induct_posint(P : Form) -> Thm:
    assert P.sort().name() == "Array"
    assert P.sort().domain() == IntSort()
    assert P.sort().range() == BoolSort()
    n = FreshConst(IntSort())
    hyp = P[0] & ForAll([n], (P[n] & n >= 0) > P[n + 1])
    #------------------------------------------
    conc =  ForAll([n], (n >= 0) > P[n])
    return trust(hyp > conc)

In [ ]:
# Open-closed induction. We can extend any point some epsilon out, and we can close any open ball back to closed.
def induct_real(P : Form) -> Thm:
    assert P.sort().name() == "Array"
    assert P.sort().domain() == RealSort()
    assert P.sort().range() == BoolSort()
    n = FreshConst(RealSort())
    eps = FreshConst(RealSort())
    hyp = P[0] & \
         ForAll([n], Exists([eps], P[n] & (eps > 0) > P[n + eps])) & \
         ForAll([n], ForAll([eps], P[x - eps] & (eps > 0) > P[x]))
    #------------------------------------------
    conc =  ForAll([n], P[n])
    return trust(hyp > conc)
# generalization to general topology?

In [40]:
";(declare-datatypes ((Nat 0)) (((zero) (succ (pred Nat)))))"
smtlib = """
;(declare-datatypes () ((Nat (zero)))) ; (succ (pred Nat)))))
(declare-datatype Nat ((zero) (succ (pred Nat))))
;(declare-datatypes ((Nat 0)) (((zero) (succ (pred Nat)))))
(declare-fun add (Nat Nat) Nat)
(declare-fun x () Nat)
(assert (forall ((x Nat)) (= (add zero x) x)))
(assert (forall ((x Nat) (y Nat)) (= (add (succ x) y) (succ (add x y)))))
(assert (let ((a!1 (and (forall ((y Nat)) (= (add zero y) (add y zero)))
                (forall ((c!13 Nat))
                  (let ((a!1 (not (forall ((y Nat))
                                    (= (add c!13 y) (add y c!13)))))
                        (a!2 (forall ((y Nat))
                               (= (add (succ x) y) (add y (succ x))))))
                    (or a!1 a!2))))))
  (or (not a!1) (forall ((c!13 Nat) (y Nat)) (= (add c!13 y) (add y c!13))))))
(assert (not (forall ((x Nat) (y Nat)) (= (add x y) (add y x)))))
(check-sat)
"""

open("/tmp/peano.smt2", "w").write(smtlib)


894

In [41]:
!vampire /tmp/peano.smt2

perf_event_open failed (instruction limiting will be disabled): Permission denied
% Running in auto input_syntax mode. Trying SMTLIB2


^C
28863 Aborted by signal SIGINT on /tmp/peano.smt2
% ------------------------------
% Version: Vampire 4.6.1 (commit af1735c99 on 2021-12-01 14:43:47 +0100)
% Linked with Z3 4.8.13.0 f03d756e086f81f2596157241e0decfb1c982299 z3-4.8.4-5390-gf03d756e0
% Termination reason: Unknown
% Termination phase: Saturation

% Memory used [KB]: 29551
% Time elapsed: 24.705 s
% ------------------------------
% ------------------------------


In [ ]:
# extending with injection axioms into the Ints

def induct(P):
  # assert P.type == Nat -> Bool ?
  n = FreshConst(Nat)
  return ForAll(n, Implies(P(Nat.zero), Forall(n, Implies(P(n), P(Nat.succ(n))))), Forall(n, P(n)))

inj = Function("inj", Nat, IntSort())
n = FreshConst(Nat)
axioms = [
  inj(Nat.zero) == 0,
  ForAll(n, inj(Nat.succ(n)) == inj(n) + 1) # recursive definition of inj
]

theorem1 = ForAll(n, inj(n) >= 0)
theorem2 = ForAll(i, Implies(i >= 0, Exist(n, inj(n) == i)))
P = lambda x: 

# HOL

So HOL is supposed to be weaker than set theory but still very expressive. 
See 
- Andrews book
- HOL Light tutorial
- Gordon paper


In [ ]:
o = BoolSort()
i = DeclareSort("i")
def EQ(A,B):


TRUE = None
FALSE = None

# Set Theory

In [44]:

# Zf
Set = DeclareSort("Set")
elem = Function("elem", Set, Set, BoolSort())
A,B,C,x,y,z = Consts("A B C x y z", Set) 

ax_emp = trust(Exists([A],ForAll([x], ~elem(B,A))))
ax_pair = trust(ForAll([A,B], Exists([C], ForAll([x], elem(x, C) == elem(x,A) | elem(x,B)))))
ax_ext = trust(ForAll([A,B], ForAll([x], elem(x,A) == elem(x,B)) == (A == B)))
ax_union = trust(ForAll([A], Exists([B], ForAll([x], elem(x,B) == Exists([y], elem(x,y) & elem(y,A))))))
def ax_sep(B,P):
    A = FreshConst(Set)
    return trust(Exists([A], ForAll([x], elem(x,A) == elem(x,B) & P(x))))

# ordered pairs

# injectivity theorem


In [ ]:
def extend(hyps,x,P, prefix):
    infer(hyps, Exists([x], P))
    s = FreshConst(Set, prefix=prefix)
    return s, trust(P.substitute(s, x))

In [ ]:
# Constructive Real

# Z3 reals are more like algebraic numbers, not reals.
CReal = ArraySort(Nat, RealSort())
eq = Function("eq", CReal, CReal, BoolSort())
ax_eq = trust(ForAll([x,y], eq(x,y) == (ForAll([n], x[n] - y[n] < 1/n))

# Hoare Logic
Separation logic. Not going to play that nice with z3.

Hmm. The raw cvc5 nterface isn't that pleasant.

In [ ]:
import cvc5
solver = cvc5.Solver()
solver.setLogic("ALL")
realSort = solver.getRealSort()
constraint1 = solver.mkTerm(Kind.LT, zero, x)
solver.assertFormula(constraint4)
solver.resetAssertions()
r2 = solver.checkSat()
unsatCore = solver.getUnsatCore();

# Category
Topos Book
Scott Encoding using exists predicate.
Well formed as precondition.

In [15]:
Arr = DeclareSort("Arr")
Ob = DeclareSort("Ob")
ex = Function("Ex", Arr, BoolSort())

f,g,h = Consts("f g h", Arr)
A,B,C = Consts("A B C", Ob)
comp = Function("comp", Arr, Arr, Arr)
id_ = Function("id", Ob, Arr)
dom = Function("dom", Arr, Ob)
cod = Function("cod", Arr, Ob)
z3.ExprRef.__lshift__ = lambda self, other: comp(self, other)
#ax_comp = trust(ForAll([f,g,h], comp(f,comp(g,h)) == comp(comp(f,g),h)))
ax_comp_assoc = trust(ForAll([f,g,h], f << (g << h) == (f << g) << h))
#ax_id_l = trust(ForAll([f,A], (cod(f) == A) > (id_(A) << f == f)))
#ax_id_r = trust(ForAll([f,A], (dom(f) == A) > (f << id_(A) == f)))
ax_id_l = trust(ForAll([f], id_(cod(f)) << f == f)) # inline the dom/cod
ax_id_r = trust(ForAll([f], f << id_(dom(f)) == f)) 
ax_cat = [ax_id_l, ax_id_r, ax_comp_assoc]

ax_dom_id = trust(ForAll([A], dom(id_(A)) == A))
ax_cod_id = trust(ForAll([A], cod(id_(A)) == A))
ax_ex_id = trust(ForAll([A], ex(id_(A))))
ax_ex_comp = trust(ForAll([f,g], ex(f << g) == (dom(f) == cod(g))))
ax_wff = [ax_dom_id, ax_cod_id, ax_ex_id, ax_ex_comp]


wff_id_id = infer(ax_wff + ax_cat, ex(id_(A) << id_(A)) & (dom(id_(A) << id_(A)) == A) & (cod(id_(A) << id_(A)) == A))
wff_id_id
#infer(ax_cat + ax_wff + [wff_id_id], id_(A) << id_(A) == id_(A))

# sanity check
infer(ax_cat + ax_wff, False)


(declare-sort Arr 0)
(declare-sort Ob 0)
(declare-fun comp (Arr Arr) Arr)
(declare-fun id (Ob) Arr)
(declare-fun cod (Arr) Ob)
(declare-fun dom (Arr) Ob)
(declare-fun Ex (Arr) Bool)
(assert (forall ((f Arr)) (= (comp (id (cod f)) f) f)))
(assert (forall ((f Arr)) (= (comp f (id (dom f))) f)))
(assert (forall ((f Arr) (g Arr) (h Arr)) (= (comp f (comp g h)) (comp (comp f g) h))))
(assert (forall ((A Ob)) (= (dom (id A)) A)))
(assert (forall ((A Ob)) (= (cod (id A)) A)))
(assert (forall ((A Ob)) (Ex (id A))))
(assert (forall ((f Arr) (g Arr)) (= (Ex (comp f g)) (= (dom f) (cod g)))))



AssertionError: unknown